In [20]:
from dotenv import load_dotenv
from langchain_chroma.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.messages.ai import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_groq import ChatGroq
from langchain_mistralai import ChatMistralAI
from langchain_openai.embeddings import OpenAIEmbeddings

In [21]:
load_dotenv()

True

## Documents

In [3]:
documents: list[Document] = [
    Document(
        metadata={"source": "mammal-pets-doc"},
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
    ),
    Document(
        metadata={"source": "mammal-pets-doc"},
        page_content="Cats are independent pets that often enjoy their own space.",
    ),
    Document(
        metadata={"source": "fish-pets-doc"},
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
    ),
    Document(
        metadata={"source": "bird-pets-doc"},
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
    ),
    Document(
        metadata={"source": "mammal-pets-doc"},
        page_content="Rabbits are social animals that need plenty of space to hop around.",
    ),
]

## Vector Stores

In [4]:
vector_store: Chroma = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())

In [11]:
# Synchronous Query
vector_store.similarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

In [12]:
# Asynchronous Query
await vector_store.asimilarity_search("cat")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

In [16]:
# Return Similarity Scores
vector_store.similarity_search_with_score("cat")

[(Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
  0.20480799674987793),
 (Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
  0.20480799674987793),
 (Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
  0.44102057814598083),
 (Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
  0.44102057814598083)]

In [18]:
# Similarity to an Embedded Query
embedding: list[float] = OpenAIEmbeddings().embed_query("cat")

vector_store.similarity_search_by_vector(embedding)

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
 Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'})]

## Retrievers

In [5]:
retriever: RunnableLambda = RunnableLambda(vector_store.similarity_search)
retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'}),
  Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'source': 'mammal-pets-doc'}),
  Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'}),
  Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'source': 'mammal-pets-doc'}),
  Document(page_content='Parrots are intelligent birds capable of mimicking human speech.', metadata={'source': 'bird-pets-doc'}),
  Document(page_content='Cats are independent pets that of

In [7]:
retriever: VectorStoreRetriever = vector_store.as_retriever(
	search_type="similarity_score_threshold",
	search_kwargs={
		"k": 1,
		"score_threshold": 0.5
	}
)
retriever.batch(["cat", "shark"])

[[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'source': 'mammal-pets-doc'})],
 [Document(page_content='Goldfish are popular pets for beginners, requiring relatively simple care.', metadata={'source': 'fish-pets-doc'})]]

In [24]:
# llm: ChatMistralAI = ChatMistralAI(model="mistral-large-latest")
llm: ChatGroq = ChatGroq(model="llama3-70b-8192")

msg: str = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt: ChatPromptTemplate = ChatPromptTemplate.from_messages([("human", msg)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [25]:
response: AIMessage = rag_chain.invoke("Tell me about cats.")
print(response.content)

According to the provided context, cats are independent pets that often enjoy their own space.
